In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
# %mpy -s {"/dev/cu.usbmodem_fs3_1"}
%mpy -s {"/dev/cu.usbmodem2101"}
# %mpy -s {"/dev/cu.usbmodem101"}

In [ ]:
!mpremote reset

In [ ]:
# %%micropython
import time
from config import config
from machine import RTC
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.time import ptp

NUM_SYNC_CYCLES = 15
TIMEOUT_MSEC = 5000

rtc = RTC()
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport._transport

# Work-around: disabling the power-saving mode on the STA_IF interface
# epn.wifi._sta.config(pm=epn.wifi._sta.PM_NONE)

miniot_network.connect()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Initial channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")
print(f"peers: {epn.epn.get_peers()}")
print(f"Active?: {epn.wifi._sta.active()}")

# Perform espnow scan to set the right channel
miniot_network.scan(timeout=750)
# epn.update_channel(5)
print(f"Final channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")

while True:
    # Perform periph sequence
    timestamps = ptp.sequence_periph(
        miniot_network.send,
        miniot_network.receive,
        lambda miniot_msg: miniot_msg.msg,
        TIMEOUT_MSEC,
        initiate_sync=True,
        num_sync_cycles=NUM_SYNC_CYCLES)

    # Calculate offsets
    offsets = [ptp.calculate_offset(t1, t2, t3, t4) for t1, t2, t3, t4 in timestamps]

    print(f"timestamps: {timestamps}")
    print(f"offsets: {offsets}")

    # Process offsets
    ave_offset = ptp.process_offsets(offsets)

    # Update time
    # rtc.offset(str(ave_offset).encode())
    rtc.offset(ave_offset)

    print(f"ave_offset: {ave_offset}")
    print(f"ave_offset bytes: {str(ave_offset).encode()}")
    print(f"now: {rtc.datetime()}")
    print(f"RTC now: {rtc.now()}")
    print(f"time.time: {time.time()}")

    time.sleep(15)


In [ ]:
# %%micropython
import network
import espnow
import time

channel = 5

sta = network.WLAN(network.WLAN.IF_STA)
sta.active(True)
sta.config(channel=channel)
print(sta.config("channel"))
print(sta.config("mac"))

e = espnow.ESPNow()
e.active(True)

peer = b'p\x04\x1d\xad|\xc0'
e.add_peer(peer)
print(e.get_peers())

for i in range(15):
    print(f"Send attempt {i}")
    e.send(peer, f"Sending on channel {channel}")
    time.sleep_ms(750)

e.send(peer, b"end")

In [ ]:
!mpremote reset